In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import *
import math as m 
tetav=5
phi=5
c=3*10**8
mp = 1.67e-27
Gamma_sh_max=100
teta=5

#JCT tu peux considérer mu comme une variable indépendante dans ton travail en cours.
#pas besoin de l'exprimer en fonction des autres variables
mu=m.sin(teta)*m.sin(tetav)*m.cos(phi)+m.cos(teta)*m.cos(tetav)
#n0=donnée
#jet structure E(teta) is given
#problème ici t!!!!

#JCT éviter de passer en arguments des constantes globales.
# Elles seront trouvées par python car définies de manière globale.
# La question de savoir si n0 est une constante ou une variable est légitime, pas mp et c
def C_BM(E, n0, mp, c):
    return np.sqrt(17 * E / (8 * np.pi * n0 * mp * c**5))
#JCT OK tu as codé exactement comme dans l'article. Maintenant réfléchis du point de vue numérique:
# 1/ tu calcules une racine carrée de C_BM. Au final est-ce vraiment nécessaire?
# 2/ tu fais une série de multiplications de constante. Ce sont des cycles de calculs perdus non? Ne vaut-il pas mieux définir une autre cst?
# 3/ Comme discuté lors de la visio, et sempiternellement en prépa, les objets de calcul en physique sont des grandeurs, elles ont des unités,
#    et donc il est vital de garder l'esprit clair sur les unités. Là l'énergie totale E va t'être fournie en erg. Est-ce que ton calcul va
#    être correct du coup?
# 4/ Dans l'expression de C_BM peux-tu faire ressortir les unités naturelles de E et de ce coefficient?

def C_ST(E, n0, mp, c):
    return (2/5) * 1.15 * (E / (n0 * mp * c**5))**(1/5)

def radius():
    def f(R):
        #JCT où est la fonction radius(t, c) ?
        mine = np.minimum((C_BM_val**2) * (T + mu * radius(t, c) / c) ** (-3) + (C_ST_val**2) * (T + mu * radius(t, c) / c) ** (-6 / 5), sh_max**2)
        return np.sqrt(mine / (1 + mine)) * c  
    
    Rp = 0
    Rs = 1
    while abs(Rp - Rs) > 10 ** (-2):
        sol = odeint(f, [0, Rs])
        Rp = Rs
        Rs = sol[0]
    return Rs

    
def shock_speed(R, C_BM_val, C_ST_val, Gamma_sh_max):
    sh_max = Gamma_sh_max
    mine=np.minimum((C_BM_val**2) * (T+mu*radius(t,c)/c)**(-3) + (C_ST_val**2) * (T+mu*radius(t,c)/c)**(-6/5), sh_max**2)
    beta_sh=np.sqrt(mine/(1+mine))
    return beta_sh


from scipy.integrate import quad

def F_nu(T, nu, mu, beta_sh, R, epsilon_0, nu_0, alpha_0, tau):
    D = 1  
    def integrand_theta(theta):
        return np.sin(theta)
    
    def integrand_phi(phi):
        return 1
    
    def integrand_numerator(theta, phi):
        return np.abs(mu - beta_sh) / ((1 - mu * beta_sh) * R**2 * (1 - beta_sh * mu)**3 * (epsilon_v/alpha_v) * (1 - np.exp(-tau * nu)))
    
    integral_theta, _ = quad(integrand_theta, 0, tetaj)
    integral_phi, _ = quad(integrand_phi, 0, 2*np.pi)
    integral_numerator, _ = quad(lambda phi: quad(lambda theta: integrand_numerator(theta, phi), 0, np.pi)[0], 0, 2*np.pi)
    
    F_nu_T = (1 / (4 * np.pi * D**2)) * integral_theta * integral_phi * integral_numerator
    
    return F_nu_T + mu * R / 3e8  # t=T+mu*R/c
#𝐷 is the luminosity distance to the source
#𝜃j is the jet truncation angle specified by a given jet structure